In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [12]:
url = 'https://fbref.com/en/squads/19538871/Manchester-United-Stats'
standings = requests.get(url)
soup = BeautifulSoup(standings.text, 'html.parser')
standings

<Response [429]>

In [11]:
standings_table = soup.select('table.stats_table')[0]
standings_table

[]

In [7]:
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
squad_links = [l for l in links if '/players/' and not ('/matchlogs/')  in l]
squad_links

NameError: name 'standings_table' is not defined

In [8]:
soup.find_all('table')

[<table class="stats_table sortable min_width" data-cols-to-freeze=",1" id="stats_standard_9"> <caption>Standard Stats <span style="color: #666; font-size:smaller">2022-2023 Manchester United: Premier League</span> Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup> <thead> <tr class="over_header"> <th aria-label="" class="over_header center" colspan="4" data-stat=""></th><th></th> <th aria-label="" class="over_header center group_start" colspan="3" data-stat="header_playing">Playing Time</th> <th aria-label="" class="over_header center group_start" colspan="8" data-stat="header_performance">Performance</th> <th aria-label="" class="over_header center group_start" colspan="4" data-stat="header_expected">Expected</th> <th aria-label="" class="over_header center group_start" colspan="3" data-stat="hea

In [9]:
matches = pd.read_html(standings.text, match="Scores & Fixtures ")[0]
matches.head(2)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2022-08-07,14:00,Premier League,Matchweek 1,Sun,Home,L,1,2,Brighton,1.4,1.5,62.0,73711.0,Harry Maguire,4-3-3,Paul Tierney,Match Report,NaN
1,2022-08-13,17:30,Premier League,Matchweek 2,Sat,Away,L,0,4,Brentford,0.9,1.6,66.0,17051.0,Harry Maguire,4-2-3-1,Stuart Attwell,Match Report,NaN


In [10]:
shooting_links = list(set([f"https://fbref.com/{l}" for l in [link.get('href') for link in soup.find_all('a')] if l and 'all_comps/shooting/' in l]))
shooting_links

['https://fbref.com//en/squads/19538871/2022-2023/matchlogs/all_comps/shooting/Manchester-United-Match-Logs-All-Competitions']

In [11]:
shooting_data = requests.get(shooting_links[0])
shooting_df = pd.read_html(shooting_data.text, match='Shooting')[0]
shooting_df.columns = shooting_df.columns.droplevel(0)
shooting_df.head(2)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2022-08-07,14:00,Premier League,Matchweek 1,Sun,Home,L,1,2,Brighton,...,15.5,0.0,0,0,1.4,1.4,0.08,-1.4,-1.4,Match Report
1,2022-08-13,17:30,Premier League,Matchweek 2,Sat,Away,L,0,4,Brentford,...,19.6,0.0,0,0,0.9,0.9,0.06,-0.9,-0.9,Match Report


In [12]:
team_df = matches.merge(shooting_df[['Date','Dist', 'Sh', 'SoT', 'FK', 'PK', 'PKatt']], on='Date', how='left')
team_df.head(2)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Dist,Sh,SoT,FK,PK,PKatt
0,2022-08-07,14:00,Premier League,Matchweek 1,Sun,Home,L,1,2,Brighton,...,4-3-3,Paul Tierney,Match Report,NaN,15.5,17.0,5.0,0.0,0.0,0.0
1,2022-08-13,17:30,Premier League,Matchweek 2,Sat,Away,L,0,4,Brentford,...,4-2-3-1,Stuart Attwell,Match Report,NaN,19.6,15.0,4.0,0.0,0.0,0.0


In [13]:
team_df.shape

(61, 25)

In [14]:
years = list(range(2023, 2019, -1))
years

[2023, 2022, 2021, 2020]

In [15]:
all_matches = []

for year in years:
    url = 'https://fbref.com/en/squads/19538871/Manchester-United-Stats'
    dt = requests.get(url)
    soup = BeautifulSoup(dt.text, 'html.parser')

    prev_season = f'{year-2}-{year-1}'
    url = f"https://fbref.com{prev_season}"

    matches = pd.read_html(dt.text, match="Scores & Fixtures ")[0]
    
    shooting_link = list(set([f"https://fbref.com{l}" for l in [link.get('href') for link in soup.find_all('a')] if l and 'all_comps/shooting/' in l]))
    shooting_data = requests.get(shooting_link[0])
    shooting_df = pd.read_html(shooting_data.text, match='Shooting')[0]
    shooting_df.columns = shooting_df.columns.droplevel(0)

    try:
        team_df = matches.merge(shooting_df[['Date','Dist', 'Sh', 'SoT', 'FK', 'PK', 'PKatt']], on='Date', how='left')
    except ValueError:
        print(f'No shooting data for {year}')
        continue

    team_df = team_df[team_df['Comp'] == 'Premier League']
    team_df['Season'] = year

    all_matches.append(team_df) 
    time.sleep(1)


In [16]:
soup.select('div.prevnext')[0]

<div class="prevnext">
<a href="/en/squads/19538871/2021-2022/Manchester-United-Stats"><div class="button2 prev">Previous Season</div></a>
</div>

In [41]:
matches_df = pd.concat(all_matches)
matches_df.head(2)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Referee,Match Report,Notes,Dist,Sh,SoT,FK,PK,PKatt,Season
0,2022-08-07,14:00,Premier League,Matchweek 1,Sun,Home,L,1,2,Brighton,...,Paul Tierney,Match Report,NaN,15.5,17.0,5.0,0.0,0.0,0.0,2023
1,2022-08-13,17:30,Premier League,Matchweek 2,Sat,Away,L,0,4,Brentford,...,Stuart Attwell,Match Report,NaN,19.6,15.0,4.0,0.0,0.0,0.0,2023


In [42]:
matches_df.shape

(152, 26)

In [43]:
# write to csv
matches_df.to_csv('../data/matches.csv', index=False)

In [44]:
import plotly.express as px

In [45]:
matches_df['Result'].dropna(inplace=True)

In [46]:
matches_df['Result'].fillna('D', inplace=True)

In [47]:
matches_df['Result'].unique()

array(['L', 'W', 'D'], dtype=object)

In [48]:
px.bar(x=matches_df['Result'].unique(), y=matches_df['Result'].value_counts().values, title='Results')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['L', 'W', 'D'], dtype=object),
              'xaxis': 'x',
              'y': array([72, 52, 28]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Results'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'y'}}}
})

In [4]:
import os

# get relative parent directory
parent_dir = os.path.dirname(os.getcwd())
parent_dir

'/home/elijah/Documents/DataProjects/manu_analysis'

In [5]:
ts = pd.read_csv(f'{parent_dir}/data/matches.csv')
ts.head(2)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Referee,Match Report,Notes,Dist,Sh,SoT,FK,PK,PKatt,Season
0,2022-08-07,14:00,Premier League,Matchweek 1,Sun,Home,L,1.0,2.0,Brighton,...,Paul Tierney,Match Report,NaN,15.5,17.0,5.0,0.0,0.0,0.0,2023
1,2022-08-13,17:30,Premier League,Matchweek 2,Sat,Away,L,0.0,4.0,Brentford,...,Stuart Attwell,Match Report,NaN,19.6,15.0,4.0,0.0,0.0,0.0,2023


In [7]:
goals_df = ts.groupby('Venue')[['GF', 'GA']].sum()
goals_df

,GF,GA
Venue,,
Away,84.0,124.0
Home,108.0,32.0


In [8]:
goals_df['GD'] = goals_df['GF'] - goals_df['GA']

In [9]:
goals_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, Away to Home
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GF      2 non-null      float64
 1   GA      2 non-null      float64
 2   GD      2 non-null      float64
dtypes: float64(3)
memory usage: 64.0+ bytes


In [10]:
goals_df.dropna(inplace=True)